# Config & Imports

In [1]:
config = {
    'data': {
        'n_datasets': 10_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 10, # the number of features
        
        'n_informative': 10,
        # The number of informative features, i.e., the number of features used to build the linear model used to generate the output.
        
        'n_targets': 1,
        # The number of regression targets, i.e., the dimension of the y output vector associated with a sample. By default, the output is a scalar.
    
        'bias': 0.0,
        # The bias term in the underlying linear model
        
        'effective_rank': 5,
        # if not None:
            # The approximate number of singular vectors required to explain most of the input data by linear combinations. Using this kind of singular spectrum in the input allows the generator to reproduce the correlations often observed in practice.
        # if None:
            # The input set is well conditioned, centered and gaussian with unit variance.
        
        'tail_strength': 0.5,
        # The relative importance of the fat noisy tail of the singular values profile if effective_rank is not None. When a float, it should be between 0 and 1.
        
        'noise': 5,
        # The standard deviation of the gaussian noise applied to the output.
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': None,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },    
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.2,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 150,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 100,
        'use_gpu': False,
        'gpu_numbers': '4',
        'RANDOM_SEED': 1,   
    }
}

In [2]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split


import os

os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_numbers'] if config['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization

import utilities

from joblib import Parallel, delayed

from IPython.display import Image


2022-05-17 15:08:46.409038: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 15:08:46.409072: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import logging
logging.getLogger('tensorflow').disabled = True

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [4]:
X_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], config['data']['n_features']])

if  config['data']['n_targets'] < 2:
    y_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], ])
    coef_list = np.zeros([config['data']['n_datasets'], config['data']['n_features'], ])
else:
    y_datasets_list = np.zeros([config['data']['n_datasets'], config['data']['n_samples'], config['data']['n_targets']])
    coef_list = np.zeros([config['data']['n_datasets'], config['data']['n_features'], config['data']['n_targets']])

In [5]:
directory = utilities.data_path_LR(config)

with open(directory + '/X_datasets_list_dataForLambda.npy', "rb") as f:
    X_datasets_list = np.load(f, allow_pickle=True)
with open(directory + '/y_datasets_list_dataForLambda.npy', "rb") as f:
    y_datasets_list = np.load(f, allow_pickle=True)
with open(directory + '/coef_list_targetForInet.npy', "rb") as f:
    coef_list = np.load(f, allow_pickle=True)

# Prepare Data (Functions)

In [6]:
def train_test_val_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=config['lambda']['data_prep']['train_test_val_split']['test_size'] + config['lambda']['data_prep']['train_test_val_split']['val_size'], 
                                                        random_state=config['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                        shuffle=config['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                        stratify=config['lambda']['data_prep']['train_test_val_split']['stratify'])
    X_test, X_val, y__test, y_val = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size=config['lambda']['data_prep']['train_test_val_split']['val_size'] / (config['lambda']['data_prep']['train_test_val_split']['test_size'] + config['lambda']['data_prep']['train_test_val_split']['val_size']), 
                                                    random_state=config['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['lambda']['data_prep']['train_test_val_split']['stratify'])
    return X_train, X_test, X_val, y_train, y_test, y_val
    

# Save Model & Metrics (functions)

In [7]:
def save_models(weights_list):
    directory = utilities.lambda_path_LR(config)
    
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    with open(directory + '/lambda_weights_list.npy', "wb") as f:
        np.save(f, weights_list, allow_pickle=True)

# Train Model

In [8]:
def train_nn(X, y, index):
    # Data Prep
    X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
                                                                          y)
    
    # Model Def
    model = Sequential()
    model.add(BatchNormalization(input_dim=config['data']['n_features']))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(config['data']['n_targets'], activation='sigmoid'))

    
    model.compile(optimizer='adam', loss='mae', metrics=['mae', keras.metrics.RootMeanSquaredError()])
    # model.summary()
    
    # Model fit
    history = model.fit(x=X_train,
                        y=y_train,
                        batch_size=config['lambda']['model_fit']['batch_size'],
                        epochs=config['lambda']['model_fit']['epochs'],
                        verbose=config['lambda']['model_fit']['verbose'],
                        callbacks=config['lambda']['model_fit']['callbacks'],
                        validation_data=(X_val, y_val),
                        shuffle=config['lambda']['model_fit']['shuffle'],
                        class_weight=config['lambda']['model_fit']['class_weight'],
                        sample_weight=config['lambda']['model_fit']['sample_weight'],
                        initial_epoch=config['lambda']['model_fit']['initial_epoch'],
                        steps_per_epoch=config['lambda']['model_fit']['steps_per_epoch'],
                        validation_steps=config['lambda']['model_fit']['validation_steps'],
                        validation_batch_size=config['lambda']['model_fit']['validation_batch_size'],
                        validation_freq=config['lambda']['model_fit']['validation_freq'],
                       )
    # print(history.history['val_loss'])
    # save_model(model, index)
    return np.concatenate([x.flatten() for x in model.get_weights()])

In [9]:
parallel = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky

weights_list = parallel(delayed(train_nn)(X_data, y_data, index) for index, (X_data, y_data) in enumerate(zip(X_datasets_list, y_datasets_list)))
                                  
del parallel

[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.
2022-05-17 15:08:53.961708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 15:08:53.961708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 15:08:53.961751: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-17 15:08:53.961754: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-17 15:08:53.979034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; d

# Inspect Metrics

In [10]:
weights_list = np.stack([np.array(x) for x in weights_list])

# Save Models

In [11]:
save_models(weights_list)

In [13]:
weights_list.shape

(10000, 7261)